In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import  LabelEncoder

<h3>Загрузка датасета</h3>

In [2]:
#загрузка датасета
df = pd.read_csv('coords_data_numerated.csv')
df = df.drop(columns=['Unnamed: 0', 'GSR'])
df.head()

,id,BreathingType,TimeStamp_sec,FirstMarkerXCoord,FirstMarkerYCoord,FirstMarkerZCoord,SecondMarkerXCoord,SecondMarkerYCoord,SecondMarkerZCoord,ThirdMarkerXCoord,ThirdMarkerYCoord,ThirdMarkerZCoord
0,1,0,0.058895,0.689,-1.453,4.417,0.881,-1.544,4.547,0.718,-1.607,4.369
1,1,0,0.111889,0.687,-1.452,4.418,0.881,-1.544,4.547,0.715,-1.607,4.371
2,1,0,0.158371,0.687,-1.451,4.419,0.881,-1.544,4.545,0.714,-1.607,4.371
3,1,0,0.262516,0.685,-1.450,4.420,0.882,-1.544,4.541,0.711,-1.608,4.373
4,1,0,0.295620,0.684,-1.449,4.420,0.883,-1.544,4.540,0.709,-1.607,4.370


In [3]:
y = df['BreathingType']
X = df.drop(columns=['BreathingType','TimeStamp_sec'])
X.tail()

,id,FirstMarkerXCoord,FirstMarkerYCoord,FirstMarkerZCoord,SecondMarkerXCoord,SecondMarkerYCoord,SecondMarkerZCoord,ThirdMarkerXCoord,ThirdMarkerYCoord,ThirdMarkerZCoord
230921,258,1.063,-1.649,3.843,1.093,-1.839,3.624,1.030,-1.867,3.859
230922,258,1.062,-1.648,3.842,1.094,-1.841,3.625,1.030,-1.867,3.857
230923,258,1.062,-1.648,3.842,1.093,-1.841,3.624,1.030,-1.868,3.855
230924,258,1.063,-1.647,3.842,1.094,-1.841,3.624,1.029,-1.868,3.853
230925,258,1.063,-1.647,3.842,1.094,-1.841,3.623,1.030,-1.868,3.853


In [4]:
#переименование столбцов
dict_renames = {
    'FirstMarkerXCoord' : 'FMX',
    'FirstMarkerYCoord' : 'FMY',
    'FirstMarkerZCoord' : 'FMZ',
    'SecondMarkerXCoord' :  'SMX',
    'SecondMarkerYCoord' : 'SMY',
    'SecondMarkerZCoord' : 'SMZ',
    'ThirdMarkerXCoord' : 'TMX',
    'ThirdMarkerYCoord' : 'TMY',
    'ThirdMarkerZCoord' : 'TMZ'
}
X = X.rename(columns=dict_renames)
X.head()

,id,FMX,FMY,FMZ,SMX,SMY,SMZ,TMX,TMY,TMZ
0,1,0.689,-1.453,4.417,0.881,-1.544,4.547,0.718,-1.607,4.369
1,1,0.687,-1.452,4.418,0.881,-1.544,4.547,0.715,-1.607,4.371
2,1,0.687,-1.451,4.419,0.881,-1.544,4.545,0.714,-1.607,4.371
3,1,0.685,-1.450,4.420,0.882,-1.544,4.541,0.711,-1.608,4.373
4,1,0.684,-1.449,4.420,0.883,-1.544,4.540,0.709,-1.607,4.370


<h3>Предобработка датасета</h3>

In [5]:
#удаление лишних временных рядов
X = X[X['id'] != 27]
X = X[X['id'] != 24]
X = X.reset_index(drop=True)
X.head()

,id,FMX,FMY,FMZ,SMX,SMY,SMZ,TMX,TMY,TMZ
0,1,0.689,-1.453,4.417,0.881,-1.544,4.547,0.718,-1.607,4.369
1,1,0.687,-1.452,4.418,0.881,-1.544,4.547,0.715,-1.607,4.371
2,1,0.687,-1.451,4.419,0.881,-1.544,4.545,0.714,-1.607,4.371
3,1,0.685,-1.450,4.420,0.882,-1.544,4.541,0.711,-1.608,4.373
4,1,0.684,-1.449,4.420,0.883,-1.544,4.540,0.709,-1.607,4.370


In [6]:
#приведение к одинаковому количеству точек для временных рядов
numeric_cols = X.drop(columns=['id'])
X2 = X.iloc[0:0]
for i in range(1, 259):
    if i in [24, 27]:
        continue
    X_id = X[X['id'] == i].reset_index(drop=True)
    for col in numeric_cols:
        X_id = X_id[:900]
        X_id = X_id.reset_index(drop=True)
    X2 = pd.concat([X2, X_id], axis=0, ignore_index=True)

In [7]:
X = X2

<h3>Вычисление новых признаков</h3>

In [8]:
#расчет новых преобразований
X2 = X.iloc[0:0]
for i in range(1, 259):
    if i in [24, 27]:
        continue
    X_id = X[X['id'] == i].reset_index(drop=True)
    for col in numeric_cols:
        X_id[col+'_roll10_mean'] = X_id[col].rolling(10).mean()
        X_id[col+'_roll10_median'] = X_id[col].rolling(10).median()
        X_id[col+'_change10'] = X_id[col].diff(periods=10)
        X_id[col+'_pct10'] = X_id[col].pct_change(periods=10)
        X_id[col+'_furier'] = np.abs(np.fft.fft(X_id[col]))

        X_id = X_id[10:]
        X_id = X_id.reset_index(drop=True)
    X2 =  pd.concat([X2, X_id], axis=0, ignore_index=True)

In [9]:
X = X2

In [11]:
# dimensions = ['X', 'Y', 'Z']
# markers = ['F', 'S', 'T']
# for dimension in dimensions:
#     markers_dimension = ['FM'+dimension, 'SM'+dimension, 'TM'+dimension]
#     X['Mean_'+dimension] = X[markers_dimension].mean(axis=1)
#     X['Median_'+dimension] = X[markers_dimension].median(axis=1)
#     X['Std_'+dimension] = X[markers_dimension].std(axis=1)
# for marker in markers:
#     markers_dimension = [marker+'MX', marker+'MY', marker+'MZ']
#     X['Mean_'+ marker] = X[markers_dimension].mean(axis=1)
#     X['Median_'+ marker] = X[markers_dimension].median(axis=1)
#     X['Std_'+marker] = X[markers_dimension].std(axis=1)

<h3>Подготовка данных для библиотеки Sktime</h3>

In [10]:
#преобразование датасета для библиотеки sktime
numeric_cols = X.drop(columns='id').columns
X_3d = []
for id in range(1, 259):
    if id in [24, 27]:
        continue
    X_id = X[X['id'] == id]
    instance_features = []
    for col in numeric_cols:
        lst = X_id[col].to_list()
        instance_features.append(lst)
    X_3d.append(instance_features)
X_3d = np.array(X_3d)

In [11]:
#преобразование датасета для библиотеки sktime
y_3d = []
for id in range(1, 259):
    if id in [24, 27]:
        continue
    df_id = df[df['id'] == id]
    y_3d.append(df_id['BreathingType'].iloc[0])
y_3d = np.array(y_3d)

In [12]:
from sklearn.metrics import recall_score, accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
#расчет метрик по тесовой выборке
def calculate_metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("F1-score: ", f1)
    report = classification_report(y_test, y_pred)
    print(report)

In [13]:
#разделение на обучающую(80%) и тестовую (20%) выборки
X_train, X_test, y_train, y_test = train_test_split(X_3d, y_3d, test_size=0.2, random_state=42)

<h3>Тестирование моделей классификации временных рядов</h3>

<h4>Нормализация данных</h4>

In [14]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_sc = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

<h4>Подготовка метрик</h4>

In [15]:
from tensorflow.keras.metrics import F1Score, Recall, Precision

metric_precision = Precision()
metric_recall = Recall()
metric_f1 = F1Score()
#metrics = ['accuracy', metric_precision, metric_recall, metric_f1]
metrics = ['accuracy', metric_f1]

<h3>MultiLayer Perceptron</h3>

In [24]:
from sktime.classification.deep_learning import MLPClassifier

model = MLPClassifier(n_epochs=20, batch_size=8, activation='softmax', verbose=True, metrics=metrics)

In [25]:
details = model.fit(X_train_sc, y_train)

C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 810, 54)]         0         
                                                                 
 flatten (Flatten)           (None, 43740)             0         
                                                                 
 dropout (Dropout)           (None, 43740)             0         
                                                                 
 dense_14 (Dense)            (None, 500)               21870500  
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_15 (Dense)            (None, 500)               250500    
                                                                 
 dropout_2 (Dropout)         (None, 500)               0   

<h3>Fully Connected Network</h3>

In [37]:
from sktime.classification.deep_learning import FCNClassifier

model = FCNClassifier(n_epochs=20, batch_size=8, activation='softmax', verbose=True, metrics=metrics)

In [38]:
details = model.fit(X_train_sc, y_train)

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 810, 54)]         0         
                                                                 
 conv1d_27 (Conv1D)          (None, 810, 128)          55424     
                                                                 
 batch_normalization_27 (Ba  (None, 810, 128)          512       
 tchNormalization)                                               
                                                                 
 activation_27 (Activation)  (None, 810, 128)          0         
                                                                 


C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


 conv1d_28 (Conv1D)          (None, 810, 256)          164096    
                                                                 
 batch_normalization_28 (Ba  (None, 810, 256)          1024      
 tchNormalization)                                               
                                                                 
 activation_28 (Activation)  (None, 810, 256)          0         
                                                                 
 conv1d_29 (Conv1D)          (None, 810, 128)          98432     
                                                                 
 batch_normalization_29 (Ba  (None, 810, 128)          512       
 tchNormalization)                                               
                                                                 
 activation_29 (Activation)  (None, 810, 128)          0         
                                                                 
 global_average_pooling1d_9  (None, 128)               0         
  (GlobalA

<h3>Long-Short Term Memory Network</h3>

In [18]:
from sktime.classification.deep_learning import LSTMFCNClassifier

#model = LSTMFCNClassifier(verbose=1, n_epochs=100, lstm_size=3, dropout=0.1)
model = LSTMFCNClassifier(verbose=1, n_epochs=100,)

In [19]:
details = model.fit(X_train_sc, y_train)

C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 810, 54)]            0         []                            
                                                                                                  
 conv1d_3 (Conv1D)           (None, 810, 128)             55424     ['input_2[0][0]']             
                                                                                                  
 batch_normalization_3 (Bat  (None, 810, 128)             512       ['conv1d_3[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 activation_3 (Activation)   (None, 810, 128)             0         ['batch_normalization_3[

<h3>Inception Time</h3>

In [2]:
from sktime.classification.deep_learning import InceptionTimeClassifier

In [18]:
model = InceptionTimeClassifier(n_epochs=20, batch_size=32, verbose=True, metrics=metrics)

In [19]:
details = model.fit(X_train_sc, y_train)

C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 810, 54)]            0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 810, 32)              1728      ['input_1[0][0]']             
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 810, 54)              0         ['input_1[0][0]']             
 D)                                                                                               
                                                                                                  
 conv1d_1 (Conv1D)           (None, 810, 32)              40960     ['conv1d[0][0]']          

<h3>MultiAttention Convolutional Neural Network</h3>

In [16]:
from sktime.classification.deep_learning import MACNNClassifier

In [17]:
model = MACNNClassifier(n_epochs=20, batch_size=8, verbose=True, metrics=metrics, activation='softmax')

In [18]:
details = model.fit(X_train_sc, y_train)

C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 810, 54)]            0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 810, 64)              10432     ['input_1[0][0]']             
                                                                                                  
 conv1d_1 (Conv1D)           (None, 810, 64)              20800     ['input_1[0][0]']             
                                                                                                  
 conv1d_2 (Conv1D)           (None, 810, 64)              41536     ['input_1[0][0]']             
                                                                                              